<a href="https://colab.research.google.com/github/kayaisacat/IAI5101_Assignment2/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part A: Supervised Learning

In [ ]:
# library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import xgboost as xgb
import tensorflow
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# using colab from drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Load the data
#df=pd.read_csv('drive/My Drive/Colab Notebooks/concrete.csv')
df=pd.read_csv('/Users/yuting/Desktop/Assignment2/concrete.csv')

In [ ]:
# check the initial five records
df.head()

In [ ]:
df.info()

## I. EDA :

### Univariate Analysis: 
#### 1. Build a histogram to show the distribution and central values of all the variables

In [ ]:
# Set up the figure
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(16, 12))

# Loop through each variable and create a histogram
for i, var in enumerate(df.columns):
    row = i // 3
    col = i % 3
    sns.histplot(data=df[var], bins=10, kde=True, ax=axs[row][col])
    axs[row][col].set_title(var)

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Get central values of all variables
central_values = df.describe().loc[['mean', '50%'], :]
central_values = central_values.T
print(central_values)

#### 2. Use a boxplot to determine if there are outliers in the variables

In [ ]:
# Set up the figure
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(16, 12))

# Loop through each variable and create a boxplot
for i, var in enumerate(df.columns):
    row = i // 3
    col = i % 3
    sns.boxplot(x=df[var], ax=axs[row][col])
    axs[row][col].set_title(var)

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

Blast Furnace Slag, Water , Super plasticizer, Fine Aggregate, Age, Compressive Strength
have outliners in dataset. 

### Multivariate Analysis:
#### 1. Use a pair plot to determine the relationship and degree of relation between independent variables and between independent variables

In [ ]:
# draw matrix
scatter_matrix(df,figsize=(20,20));

#### 2. Use a heatmap to check for correlation between predictor variables

In [ ]:
corr = df.corr()
corr

In [ ]:
plt.figure(figsize=(15,12))
sns.heatmap(corr, annot = True)

## II. Feature Engineering

### Ensure data is in the correct format for downstream processes
#### 1. Check for duplicates & missing values and drop, if present

In [ ]:
# check for missing values
print("Missing values:\n", df.isnull().sum())

In [ ]:
# check for deplicate values
print("Duplicates:\n", df.duplicated().sum())

In [ ]:
# the shape of dataframe
df.shape

In [ ]:
# drop duplicates
df = df.drop_duplicates()
# check the data shape
df.shape

There are 0 missing value and 25 duplicate values in the concrete dataset,  after drop the total data changed from 1030 rows to 1005 rows.

#### 2. Remove possible outliners in the dataset


I define the outliners as greater than 75% quartile add the 1.5 Interquatrile range and less than 25% quartile subtract the 1.5 Interquatrile range.

In [ ]:
# Drop the outlier observations using quartile
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
iqr = q3 - q1
lower_threshold = q1 - (1.5 * iqr)
upper_threshold = q3 + (1.5 * iqr)

# Identify the outlier observations
outliers_low = (df < lower_threshold)
outliers_high = (df > upper_threshold)
outliers = outliers_low | outliers_high

# Remove the outlier observations
df = df[~outliers.any(axis=1)]

In [ ]:
# check the outliers have been removed
# Set up the figure
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(16, 12))

# Loop through each variable and create a boxplot
for i, var in enumerate(df.columns):
    row = i // 3
    col = i % 3
    sns.boxplot(x=df[var], ax=axs[row][col])
    axs[row][col].set_title(var)

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

As you can see from boxplots, unreasonable outliers have been removed. For example, there are four values ​​in age that are over 150 years old. Although there are outliers in the above image, they are all within a reasonable range, so there is no need to deal with them again.

#### 3. Check for zero in the dataset and impute with the mean

In [ ]:
print('Frequency of negative or zero is:\n', (df <= 0).sum())

In [ ]:
# Check for zeros in the dataset and impute with the mean
imp_mean = SimpleImputer(missing_values=0, strategy='mean')
df = pd.DataFrame(imp_mean.fit_transform(df), columns=df.columns)
print('Frequency of negative or zero is:\n', (df <= 0).sum())

#### 4. Check for class imblance and handle, if necessary

In [ ]:
# The target variable is CompressiveStrength
# so we will check for class imblance and handle by examining the distribution 
sns.histplot(df['CompressiveStrength'], kde=True)
plt.tight_layout(); plt.show()


From the histgoram, we can see that the distribution is not heavily skewed towards one class, so we can say that there is no class imblance.

#### 5. Scale the data using a standard scaler

In [ ]:
# Initialize the StandardScaler 
scaler = StandardScaler() 
# To scale data sicne all data are not binary
df_scaled = scaler.fit_transform(df)

## III. Model Development I
### Ensemble Method:

#### 1. Split dataset into train (70%) and test (30%) and build predictive models to determine the strength of concrete using the following techniques: K-Nearest Neighbor Regressor, Random Forest, and XGBoost

In [ ]:
# Split the data into training and test sets
X = df.drop(['CompressiveStrength'], axis=1)
y = df['CompressiveStrength']

# set seed for reproducability
np.random.seed(100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape,y_train.shape)

# Scale the data using a standard scaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# K-Nearest Neighbor Regressor
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
knn_score = knn.score(X_test_scaled, y_test)
print('K-Nearest Neighbor Regressor Score:', knn_score)

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, max_depth=10)
rf.fit(X_train_scaled, y_train)
rf_score = rf.score(X_test_scaled, y_test)
print("Random Forest Regressor score:", rf_score)

# XGBoost Regressor
xgb = xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1)
xgb.fit(X_train_scaled, y_train)
xgb_score = xgb.score(X_test_scaled, y_test)
print("XGBoost Regressor score:", xgb_score)

We got the evaluation score of each model, and we can see that the XGBoost is better perfomance than others. The R-squared is 0.93, which means XGBoost has a great prediction.

#### 2. Use a voting regressor to predict the values for the ensemble of heterogeneous models above (K-Nearest Neighbor Regressor, Random Forest & xGBoost).

In [ ]:
# Define the voting regressor
voting_regressor = VotingRegressor([('knn', knn), ('rf', rf), ('xgb', xgb)])

# Train the voting regressor
voting_regressor.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = voting_regressor.predict(X_test_scaled)

# Evaluate the performance of the voting regressor
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Evaluate the performance of the voting regressor
score = voting_regressor.score(X_test_scaled, y_test)

print("MSE:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)
print("Score:", score)


The ensemble of heterogeneous models have a good predition, and the evaluation of the voting regressor has a great performance which R-squared equals to 0.9.

## IV. Model Development II
### Deep Learning:

#### 1. Split dataset into train (70%) and test (30%) and train a deep neural network using Keras.

#### 2. Try to improve the model by changing the activation function or dropout rate. What effects does any of these have on the result?

## V. Model Comparsion, Evaluation

# Part B: Unsupervised Learning